# Reti generative

Le Reti Neurali Ricorrenti (RNN) e le loro varianti con celle gated, come le celle Long Short Term Memory (LSTM) e le Gated Recurrent Units (GRU), hanno fornito un meccanismo per il modellamento del linguaggio, ovvero possono apprendere l'ordine delle parole e fornire previsioni per la parola successiva in una sequenza. Questo ci permette di utilizzare le RNN per **compiti generativi**, come la generazione di testo ordinario, la traduzione automatica e persino la descrizione di immagini.

Nell'architettura RNN che abbiamo discusso nell'unità precedente, ogni unità RNN produceva il prossimo stato nascosto come output. Tuttavia, possiamo anche aggiungere un altro output a ciascuna unità ricorrente, che ci consentirebbe di generare una **sequenza** (che ha la stessa lunghezza della sequenza originale). Inoltre, possiamo utilizzare unità RNN che non accettano un input a ogni passo, ma prendono solo un vettore di stato iniziale e poi producono una sequenza di output.

In questo notebook, ci concentreremo su modelli generativi semplici che ci aiutano a generare testo. Per semplicità, costruiamo una rete **a livello di carattere**, che genera testo lettera per lettera. Durante l'addestramento, dobbiamo prendere un corpus di testo e suddividerlo in sequenze di lettere.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset,test_dataset,classes,vocab = load_dataset()

Loading dataset...
Building vocab...


## Costruire un vocabolario di caratteri

Per creare una rete generativa a livello di caratteri, è necessario suddividere il testo in singoli caratteri anziché in parole. Questo può essere fatto definendo un tokenizer diverso:


In [2]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(char_tokenizer(line))
vocab = torchtext.vocab.vocab(counter)

vocab_size = len(vocab)
print(f"Vocabulary size = {vocab_size}")
print(f"Encoding of 'a' is {vocab.get_stoi()['a']}")
print(f"Character with code 13 is {vocab.get_itos()[13]}")

Vocabulary size = 82
Encoding of 'a' is 1
Character with code 13 is c


Vediamo l'esempio di come possiamo codificare il testo dal nostro dataset:


In [3]:
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

enc(train_dataset[0][1])

tensor([ 0,  1,  2,  2,  3,  4,  5,  6,  3,  7,  8,  1,  9, 10,  3, 11,  2,  1,
        12,  3,  7,  1, 13, 14,  3, 15, 16,  5, 17,  3,  5, 18,  8,  3,  7,  2,
         1, 13, 14,  3, 19, 20,  8, 21,  5,  8,  9, 10, 22,  3, 20,  8, 21,  5,
         8,  9, 10,  3, 23,  3,  4, 18, 17,  9,  5, 23, 10,  8,  2,  2,  8,  9,
        10, 24,  3,  0,  1,  2,  2,  3,  4,  5,  9,  8,  8,  5, 25, 10,  3, 26,
        12, 27, 16, 26,  2, 27, 16, 28, 29, 30,  1, 16, 26,  3, 17, 31,  3, 21,
         2,  5,  9,  1, 23, 13, 32, 16, 27, 13, 10, 24,  3,  1,  9,  8,  3, 10,
         8,  8, 27, 16, 28,  3, 28,  9,  8,  8, 16,  3,  1, 28,  1, 27, 16,  6])

## Addestrare un RNN generativo

Il modo in cui addestreremo l'RNN per generare testo è il seguente. A ogni passo, prenderemo una sequenza di caratteri di lunghezza `nchars` e chiederemo alla rete di generare il carattere successivo per ogni carattere di input:

![Immagine che mostra un esempio di generazione RNN della parola 'HELLO'.](../../../../../translated_images/it/rnn-generate.56c54afb52f9781d.webp)

A seconda dello scenario specifico, potremmo anche voler includere alcuni caratteri speciali, come *fine-sequenza* `<eos>`. Nel nostro caso, vogliamo semplicemente addestrare la rete per una generazione di testo continua, quindi fisseremo la dimensione di ogni sequenza uguale a `nchars` token. Di conseguenza, ogni esempio di addestramento sarà composto da `nchars` input e `nchars` output (che sono la sequenza di input spostata di un simbolo a sinistra). Il minibatch sarà composto da diverse di queste sequenze.

Il modo in cui genereremo i minibatch sarà prendere ogni testo di notizia di lunghezza `l` e generare tutte le possibili combinazioni input-output da esso (ci saranno `l-nchars` tali combinazioni). Queste formeranno un minibatch, e la dimensione dei minibatch sarà diversa a ogni passo di addestramento.


In [4]:
nchars = 100

def get_batch(s,nchars=nchars):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

get_batch(train_dataset[0][1])

(tensor([[ 0,  1,  2,  ..., 28, 29, 30],
         [ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         ...,
         [20,  8, 21,  ...,  1, 28,  1],
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16]]),
 tensor([[ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         [ 2,  3,  4,  ...,  1, 16, 26],
         ...,
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16],
         [ 5,  8,  9,  ..., 27, 16,  6]]))

Ora definiamo la rete generativa. Può essere basata su qualsiasi cella ricorrente di cui abbiamo discusso nell'unità precedente (semplice, LSTM o GRU). Nel nostro esempio utilizzeremo LSTM.

Poiché la rete prende caratteri come input e la dimensione del vocabolario è piuttosto piccola, non abbiamo bisogno di uno strato di embedding; l'input codificato in one-hot può essere inviato direttamente alla cella LSTM. Tuttavia, poiché passiamo numeri di caratteri come input, dobbiamo codificarli in one-hot prima di passarli all'LSTM. Questo viene fatto chiamando la funzione `one_hot` durante il passaggio `forward`. L'encoder di output sarà uno strato lineare che convertirà lo stato nascosto in un output codificato in one-hot.


In [5]:
class LSTMGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.rnn = torch.nn.LSTM(vocab_size,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, s=None):
        x = torch.nn.functional.one_hot(x,vocab_size).to(torch.float32)
        x,s = self.rnn(x,s)
        return self.fc(x),s

Durante l'addestramento, vogliamo essere in grado di campionare testo generato. Per fare ciò, definiremo la funzione `generate` che produrrà una stringa di output di lunghezza `size`, partendo dalla stringa iniziale `start`.

Il funzionamento è il seguente. Per prima cosa, passeremo l'intera stringa iniziale attraverso la rete, ottenendo lo stato di output `s` e il prossimo carattere previsto `out`. Poiché `out` è codificato in one-hot, utilizziamo `argmax` per ottenere l'indice del carattere `nc` nel vocabolario e usiamo `itos` per determinare il carattere effettivo, aggiungendolo alla lista risultante di caratteri `chars`. Questo processo di generazione di un carattere viene ripetuto `size` volte per generare il numero richiesto di caratteri.


In [8]:
def generate(net,size=100,start='today '):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            nc = torch.argmax(out[0][-1])
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)

Ora iniziamo l'addestramento! Il ciclo di addestramento è quasi identico a quello di tutti i nostri esempi precedenti, ma invece di mostrare l'accuratezza, stampiamo il testo generato campionato ogni 1000 epoche.

Bisogna prestare particolare attenzione al modo in cui calcoliamo la perdita. Dobbiamo calcolare la perdita utilizzando l'output codificato in one-hot `out` e il testo previsto `text_out`, che è la lista degli indici dei caratteri. Fortunatamente, la funzione `cross_entropy` si aspetta come primo argomento l'output non normalizzato della rete e come secondo argomento il numero della classe, che è esattamente ciò che abbiamo. Inoltre, esegue automaticamente la media sulla dimensione del minibatch.

Limitiamo anche l'addestramento a un numero di campioni pari a `samples_to_train`, per evitare di aspettare troppo a lungo. Ti incoraggiamo a sperimentare e provare un addestramento più lungo, possibilmente per diverse epoche (in tal caso, dovresti creare un altro ciclo attorno a questo codice).


In [9]:
net = LSTMGenerator(vocab_size,64).to(device)

samples_to_train = 10000
optimizer = torch.optim.Adam(net.parameters(),0.01)
loss_fn = torch.nn.CrossEntropyLoss()
net.train()
for i,x in enumerate(train_dataset):
    # x[0] is class label, x[1] is text
    if len(x[1])-nchars<10:
        continue
    samples_to_train-=1
    if not samples_to_train: break
    text_in, text_out = get_batch(x[1])
    optimizer.zero_grad()
    out,s = net(text_in)
    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
    loss.backward()
    optimizer.step()
    if i%1000==0:
        print(f"Current loss = {loss.item()}")
        print(generate(net))

Current loss = 4.398899078369141
today sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr s
Current loss = 2.161320447921753
today and to the tor to to the tor to to the tor to to the tor to to the tor to to the tor to to the tor t
Current loss = 1.6722588539123535
today and the court to the could to the could to the could to the could to the could to the could to the c
Current loss = 2.423795223236084
today and a second to the conternation of the conternation of the conternation of the conternation of the 
Current loss = 1.702607274055481
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.692358136177063
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.9722288846969604
today and the control the control the control the control the control the control the control the control 
Current loss = 1.8

Questo esempio genera già un testo piuttosto buono, ma può essere ulteriormente migliorato in diversi modi:
* **Migliore generazione di minibatch**. Il modo in cui abbiamo preparato i dati per l'addestramento è stato generare un minibatch da un singolo campione. Questo non è ideale, perché i minibatch hanno dimensioni diverse e alcuni di essi non possono nemmeno essere generati, poiché il testo è più piccolo di `nchars`. Inoltre, minibatch di piccole dimensioni non sfruttano sufficientemente la GPU. Sarebbe più saggio ottenere un grande blocco di testo da tutti i campioni, poi generare tutte le coppie input-output, mescolarle e generare minibatch di dimensioni uguali.
* **LSTM multilivello**. Ha senso provare 2 o 3 livelli di celle LSTM. Come abbiamo menzionato nell'unità precedente, ogni livello di LSTM estrae determinati schemi dal testo e, nel caso di un generatore a livello di carattere, possiamo aspettarci che il livello inferiore di LSTM sia responsabile dell'estrazione delle sillabe, mentre i livelli superiori - delle parole e delle combinazioni di parole. Questo può essere implementato semplicemente passando il parametro del numero di livelli al costruttore di LSTM.
* Potresti anche voler sperimentare con **unità GRU** per vedere quali offrono prestazioni migliori, e con **dimensioni diverse dei livelli nascosti**. Un livello nascosto troppo grande potrebbe portare a overfitting (ad esempio, la rete apprenderà il testo esatto), mentre una dimensione troppo piccola potrebbe non produrre risultati soddisfacenti.


## Generazione di testo morbido e temperatura

Nella precedente definizione di `generate`, sceglievamo sempre il carattere con la probabilità più alta come prossimo carattere nel testo generato. Questo portava spesso al fatto che il testo "ciclasse" tra le stesse sequenze di caratteri ripetutamente, come in questo esempio:
```
today of the second the company and a second the company ...
```

Tuttavia, se osserviamo la distribuzione di probabilità per il prossimo carattere, potrebbe accadere che la differenza tra alcune delle probabilità più alte non sia così grande, ad esempio un carattere potrebbe avere una probabilità di 0,2, un altro di 0,19, ecc. Per esempio, quando cerchiamo il prossimo carattere nella sequenza '*play*', il carattere successivo potrebbe essere ugualmente uno spazio o **e** (come nella parola *player*).

Questo ci porta alla conclusione che non è sempre "giusto" selezionare il carattere con la probabilità più alta, perché scegliere il secondo più probabile potrebbe comunque portarci a un testo significativo. È più saggio **campionare** i caratteri dalla distribuzione di probabilità fornita dall'output della rete.

Questo campionamento può essere effettuato utilizzando la funzione `multinomial`, che implementa la cosiddetta **distribuzione multinomiale**. Una funzione che implementa questa generazione di testo **morbida** è definita di seguito:


In [10]:
def generate_soft(net,size=100,start='today ',temperature=1.0):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            #nc = torch.argmax(out[0][-1])
            out_dist = out[0][-1].div(temperature).exp()
            nc = torch.multinomial(out_dist,1)[0]
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start='Today ',temperature=i)}\n")

--- Temperature = 0.3
Today and a company and complete an all the land the restrational the as a security and has provers the pay to and a report and the computer in the stand has filities and working the law the stations for a company and with the company and the final the first company and refight of the state and and workin

--- Temperature = 0.8
Today he oniis its first to Aus bomblaties the marmation a to manan  boogot that pirate assaid a relaid their that goverfin the the Cappets Ecrotional Assonia Cition targets it annight the w scyments Blamity #39;s TVeer Diercheg Reserals fran envyuil that of ster said access what succers of Dour-provelith

--- Temperature = 1.0
Today holy they a 11 will meda a toket subsuaties, engins for Chanos, they's has stainger past to opening orital his thempting new Nattona was al innerforder advan-than #36;s night year his religuled talitatian what the but with Wednesday to Justment will wemen of Mark CCC Camp as Timed Nae wome a leaders

--- Temper

Abbiamo introdotto un altro parametro chiamato **temperatura**, che viene utilizzato per indicare quanto strettamente dovremmo aderire alla probabilità più alta. Se la temperatura è 1.0, facciamo un campionamento multinomiale equo, e quando la temperatura va all'infinito - tutte le probabilità diventano uguali, e selezioniamo casualmente il prossimo carattere. Nell'esempio qui sotto possiamo osservare che il testo diventa privo di senso quando aumentiamo troppo la temperatura, e somiglia a un testo "ciclato" generato rigidamente quando si avvicina a 0.



---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione AI [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa dovrebbe essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale effettuata da un esperto umano. Non siamo responsabili per eventuali fraintendimenti o interpretazioni errate derivanti dall'uso di questa traduzione.
